In [1]:
import cv2
import os
import random
import numpy as np
from matplotlib import pyplot as plt
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, Conv2D, Dense, MaxPooling2D, Input, Flatten,Subtract, Lambda, Concatenate
import tensorflow as tf
from tensorflow.keras.metrics import Precision, Recall, Accuracy

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
SIZE = 200

In [19]:

def get_class_file_paths(path):
    classes = os.listdir(path)
    file_paths = {}
    for class_name in classes:
        class_path = os.path.join(path, class_name)
        file_paths[class_name] = tf.data.Dataset.list_files(class_path + '/*.jpg')
    return file_paths

In [5]:
def preprocess(file_path):

    byte_img = tf.io.read_file(file_path)
    img = tf.io.decode_jpeg(byte_img)

    img = tf.image.resize(img, (SIZE,SIZE))
    img = img / 255.0

    return img

In [6]:
def preprocess_twin(input_img, validation_img, label):
    return (preprocess(input_img), preprocess(validation_img), label)


In [7]:
def make_embedding():
    inp = Input(shape=(SIZE,SIZE,1), name='input_image')

    c1 = Conv2D(64, (10,10), activation='relu')(inp)
    m1 = MaxPooling2D(64, (2,2), padding='same')(c1)

    c2 = Conv2D(128, (7,7), activation='relu')(m1)
    m2 = MaxPooling2D(64, (2,2), padding='same')(c2)

    c3 = Conv2D(128, (4,4), activation='relu')(m2)
    m3 = MaxPooling2D(64, (2,2), padding='same')(c3)

    c4 = Conv2D(256, (4,4), activation='relu')(m3)
    f1 = Flatten()(c4)
    d1 = Dense(4096, activation='sigmoid')(f1)


    return Model(inputs=[inp], outputs=[d1], name='embedding')

In [8]:
embedding = make_embedding()

In [9]:
# Siamese L1 Distance class
class L1Dist(Layer):

    def __init__(self, **kwargs):
        super().__init__()

    def call(self, input_embedding, validation_embedding):
        return tf.math.abs(input_embedding - validation_embedding)

In [ ]:
# Reload model
siamese_model = tf.keras.models.load_model('/content/drive/MyDrive/gp/model/siamesemodel_gray_cnn.h5',
                                   custom_objects={'L1Dist':L1Dist, 'BinaryCrossentropy':tf.losses.BinaryCrossentropy})

In [13]:
def verify(model, detection_threshold, verification_threshold):
    results = []
    for image in os.listdir(os.path.join('/content/drive/MyDrive/gp/gray_inf/veryfication')):
        input_img = preprocess(os.path.join('/content/drive/MyDrive/gp/gray_inf/', '7a.jpg'))
        validation_img = preprocess(os.path.join('/content/drive/MyDrive/gp/gray_inf/veryfication', image))
        print(image)

        result = model.predict(list(np.expand_dims([input_img, validation_img], axis=1)))
        results.append(result)

    detection = np.sum(np.array(results) > detection_threshold)

    verification = detection / len(os.listdir(os.path.join('/content/drive/MyDrive/gp/gray_inf/veryfication')))
    verified = verification > verification_threshold

    return results, verified

In [16]:
results, verified = verify(siamese_model, 0.9,0.9)

1b.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
2b.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
4b.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
6b.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
4a.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
2a.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
5b.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
7b.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
6a.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
5a.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
3b.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
3a.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step


In [17]:
results

[array([[1.]], dtype=float32),
 array([[1.]], dtype=float32),
 array([[1.]], dtype=float32),
 array([[1.]], dtype=float32),
 array([[1.]], dtype=float32),
 array([[1.]], dtype=float32),
 array([[1.]], dtype=float32),
 array([[1.]], dtype=float32),
 array([[1.]], dtype=float32),
 array([[1.]], dtype=float32),
 array([[1.]], dtype=float32),
 array([[1.]], dtype=float32)]

In [18]:
verified

True